In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


# Generative Adversarial Network 1D normal distribution Example

본 실습은 딥러닝 라이브러리 PyTorch를 기반으로 진행

실습에 필요한 라이브러리 불러오기

In [0]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from torch.autograd import Variable

PyTorch가 GPU를 통해 연산을 하는지 확인하기( True가 나올 때, 연산은 GPU에서 적용)

In [0]:
torch.cuda.is_available()

True

Data Input과 관련한 함수들을 정의

In [0]:
(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)

In [0]:
print("Using data [%s]" % (name))

Using data [Data and variances]


정규분포(mu, sigma)에서 n개의 sample 뽑기

In [0]:
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n))) 

0과 1사이의 숫자를 임의로 생성 - 정규분포가 아님

In [0]:
def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)

Generator class 생성- PyTorch의 대부분은 class method로 구성되어 있음

In [0]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        
        return self.map3(x)

Discrminator class 생성

In [0]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        return F.sigmoid(self.map3(x))

기타 필요한 함수들 정의

In [0]:
def extract(v):
    return v.data.storage().tolist()

In [0]:
def stats(d):
    return [round(np.mean(d), 4), round(np.std(d), 4)]

In [0]:
def decorate_with_diffs(data, exponent):

    mean = torch.mean(data.data, 1)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0])

    diffs = torch.pow(data - Variable(mean_broadcast), exponent)

    return torch.cat([data, diffs], 1)

###### Hyperparameter 정의

----------> 생성하고자 하는 1D Normal Distribution

In [0]:
data_mean = 4
data_stddev = 1.25

----------> Generator Network 구조 정의

In [0]:
g_input_size = 1     
g_hidden_size = 50   # G의 레이어 크기
g_output_size = 1    # G의 결과값의 크기 : (1,N) - 위조 지폐의 크기

----------> Discriminator Network 구조 정의

In [0]:
d_input_size = 100   # Minibatch size
d_hidden_size = 50   # D의 레이어 크기
d_output_size = 1    # 진짜인지 가짜인지의 여부(확률값) : (1,1)
minibatch_size = d_input_size

----------> Generator/ Discriminator Sample 생성

In [0]:
gi_sampler = get_generator_input_sampler()

In [0]:
d_sampler = get_distribution_sampler(data_mean, data_stddev)

In [0]:
d_steps,g_steps = 1, 1

----------> d_learning_rate : Discriminator 학습율 지정/ g_learning_rate : Generator 학습율 지정

----------> 해당 값들을 너무 크게 지정하거나, 너무 작게 지정하면 학습에 문제가 발생

(크게 지정한 경우:학습이 되지 않는 상황 발생/ 작게 지정한 경우 : 학습 속도가 매우 더딤)

In [0]:
d_learning_rate = 2e-4
g_learning_rate = 2e-4
optim_betas = (0.9, 0.999)

----------> Epoch 수 지정 및 결과 횟수 간격 지정

In [0]:
num_epochs = 2600 # 얼마나 최적화할지
print_interval = 10 # 몇번마다 결과를 출력할지

----------> Generator/ Discriminator Network Instance 생성

In [0]:
G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
D = Discriminator(input_size=d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)

----------> 진짜 데이터이냐, 가짜 데이터(생성된 데이터)냐? = 2개의 Class를 분류하는 Classification 모델 필요

In [0]:
criterion = nn.BCELoss()

---------> Optimizer를 정의

In [0]:
d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)

### ---------> 학습 시작

In [0]:
for epoch in range(num_epochs):
    
    if 'img' not in os.listdir():
        os.mkdir("img")
        
    for d_index in range(d_steps):
        # 1. Train D
        D.zero_grad()

        #  1A: 실제 데이터로 D 학습
        d_real_data = Variable(d_sampler(d_input_size))
        d_real_decision = D(preprocess(d_real_data))
        d_real_error = criterion(d_real_decision, Variable(torch.ones(1)))  # 1 = 실제 데이터
        d_real_error.backward() 

        #  1B: 가짜 데이터로 D 학습
        d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
        d_fake_data = G(d_gen_input).detach()  
        d_fake_decision = D(preprocess(d_fake_data.t()))
        d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(1)))  # 0 = 생성/가짜 데이터
        d_fake_error.backward()
        d_optimizer.step()     # Discriminator 의 파라미터 변화

    for g_index in range(g_steps):

        # 2. D의 반응에 따라 G학습

        # G 초기화
        G.zero_grad()

        gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
        g_fake_data = G(gen_input)
        dg_fake_decision = D(preprocess(g_fake_data.t()))
        g_error = criterion(dg_fake_decision, Variable(torch.ones(1)))  # 생성데이터가 실제 데이터인것 처럼 생성하기를 원함

        g_error.backward()
        g_optimizer.step()  # Generator의 파라미터 변화

    if (epoch+1) % (print_interval*10) == 0:
        print("%s: D: %.4f/%.4f G: %.4f (Real: %s, Fake: %s) \n" % (epoch+1,
                                                            round(extract(d_real_error)[0],4),
                                                            round(extract(d_fake_error)[0],4),
                                                            round(extract(g_error)[0],4),
                                                            stats(extract(d_real_data)),
                                                            stats(extract(d_fake_data))
                                                                   )
             )
    if (epoch+1) % print_interval == 0:
        [mu_real, sigma_real] = stats(extract(d_real_data))
        [mu_fake, sigma_fake] = stats(extract(d_fake_data))

        x1 = np.linspace(mu_real-9*sigma_real,mu_real+9*sigma_real, 100)
        x2 = np.linspace(mu_fake-9*sigma_fake,mu_real+9*sigma_fake, 100)
        plt.plot(x1,mlab.normpdf(x1, mu_real, sigma_real))
        plt.plot(x2,mlab.normpdf(x2, mu_fake, sigma_fake))
        plt.title('Generate 1D Gaussian Distribution using GAN: %7d epoch'%(epoch+1))
        plt.xlabel('Data values')
        plt.ylabel('Probability density')
        plt.savefig('img/Generate 1D Gaussian Distribution using GAN: %7d epoch'%(epoch+1) + '.png',dpi=200)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.